In [1]:
#BeautifulSoup
from bs4 import BeautifulSoup
import requests

import re
import pandas as pd

In [2]:
url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?tt=1&at=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50"

In [3]:
all_df = []

for page in range(1,501):
    
    req = requests.get(url + "&p=" + str(page))
    soup = BeautifulSoup(req.text, "html.parser")
    
    price = [price.get_text() for price in soup.find_all("div", class_='ergov3-priceannonce')]
    good_prices = [prices.strip() for prices in price]
    
    cities = [city.get_text() for city in soup.find_all("cite")]
    good_cities = [cite.strip() for cite in cities]
    
    annonceur = [anno.get_text() for anno in soup.find_all("div", class_='ergov3-bottomannonce ergov3-bottomannonce_2lines')]
    good_annonceur = [annonc.strip() for annonc in annonceur]
    
    text_annonce = [annonce.get_text() for annonce in soup.find_all("p", class_='txt-long')]
    good_text_annonce = [gta.strip() for gta in text_annonce]
    
    title_liste = []
    for title in soup.select('div.ergov3-h3'):
        title_liste.append(title.span.text)
    good_title_liste = [gtl.strip().replace("\s+", " ") for gtl in title_liste]
    
    all_df.append(pd.DataFrame({'Title': good_title_liste, 'Prix': good_prices,'Ville': good_cities, 'Annonceur': good_annonceur, 'Description': good_text_annonce}))
    

out = pd.concat(all_df, ignore_index = True)

In [5]:
out.head()

,Title,Prix,Ville,Annonceur,Description
0,"Appartement\r\n1\r\npièce,\t\t\t\t\t\t\t\t\t\t...",260 000 €,Alfortville (94140),Professionnel\n\n\n\nBONAPARTE PROMOTION ILE D...,Superbe studio spacieux et fonctionnel hyper c...
1,"Maison\r\n3\r\npièces,\t\t\t\t\t\t\t\t\t\t\t\t...",140 000 €,Beton-Bazoches (77320),Professionnel\n\n\n\nIMKIZ\r\n80 annonces\t\t\...,Maison familiale 3 pièces avec cheminée. maiso...
2,"Maison\r\n4\r\npièces,\t\t\t\t\t\t\t\t\t\t\t\t...",222 000 €,La Ferté-sous-Jouarre (77260),Professionnel\n\n\n\nAGENCE DE LA VALLEE\r\n44...,"Atypique maison briarde, logement à l'étage av..."
3,"Appartement\r\n3\r\npièces,\t\t\t\t\t\t\t\t\t\...",475 000 €,Bezons (95870),Professionnel\n\n\n\nIMKIZ\r\n80 annonces\t\t\...,Appartement 3 pièces. appartement t3 récent de...
4,"Maison\r\n7\r\npièces,\t\t\t\t\t\t\t\t\t\t\t\t...",1 200 000 €,La Celle-Saint-Cloud (78170),Professionnel\n\n\n\nLAFORET LE CHESNAY\r\n52 ...,Maison la celle saint cloud 7 pièce(s) 185 m2....


In [6]:
out = out.replace(r'\r+|\n+|\t+',' ', regex = True)

In [7]:
out.head()

,Title,Prix,Ville,Annonceur,Description
0,"Appartement 1 pièce, 35 m²",260 000 €,Alfortville (94140),Professionnel BONAPARTE PROMOTION ILE DE FRANC...,Superbe studio spacieux et fonctionnel hyper c...
1,"Maison 3 pièces, 66 m²",140 000 €,Beton-Bazoches (77320),Professionnel IMKIZ 80 annonces Voir l'annonce,Maison familiale 3 pièces avec cheminée. maiso...
2,"Maison 4 pièces, 179 m²",222 000 €,La Ferté-sous-Jouarre (77260),Professionnel AGENCE DE LA VALLEE 44 annonces...,"Atypique maison briarde, logement à l'étage av..."
3,"Appartement 3 pièces, 70 m²",475 000 €,Bezons (95870),Professionnel IMKIZ 80 annonces Voir l'annonce,Appartement 3 pièces. appartement t3 récent de...
4,"Maison 7 pièces, 185 m²",1 200 000 €,La Celle-Saint-Cloud (78170),Professionnel LAFORET LE CHESNAY 52 annonces ...,Maison la celle saint cloud 7 pièce(s) 185 m2....


In [8]:
out['Bien'] = out.Title.str.extract(r'(\S+)', expand=False)
out["Pieces"] = out["Title"].str.extract(r'(\d+)\s*pièces?', expand=False)
out['Taille'] = out["Title"].str.extract(r'(\d+(?:\.\d+)?)\s*m²', expand=False)
out["Annonceur"] = out["Annonceur"].str.extract(r'(^[a-zA-Z]+)', expand=False)
out["Prix"] = out["Prix"].replace(r'€','', regex = True)
out["Code_Postal"] = out["Ville"].str.extract(r'(\d+)', expand=False)
out["Ville"] = out["Ville"].str.extract(r'([^0-9]+)\s\(\d+\)', expand=False)

In [9]:
out.head()

,Title,Prix,Ville,Annonceur,Description,Bien,Pieces,Taille,Code_Postal
0,"Appartement 1 pièce, 35 m²",260 000,Alfortville,Professionnel,Superbe studio spacieux et fonctionnel hyper c...,Appartement,1,35,94140
1,"Maison 3 pièces, 66 m²",140 000,Beton-Bazoches,Professionnel,Maison familiale 3 pièces avec cheminée. maiso...,Maison,3,66,77320
2,"Maison 4 pièces, 179 m²",222 000,La Ferté-sous-Jouarre,Professionnel,"Atypique maison briarde, logement à l'étage av...",Maison,4,179,77260
3,"Appartement 3 pièces, 70 m²",475 000,Bezons,Professionnel,Appartement 3 pièces. appartement t3 récent de...,Appartement,3,70,95870
4,"Maison 7 pièces, 185 m²",1 200 000,La Celle-Saint-Cloud,Professionnel,Maison la celle saint cloud 7 pièce(s) 185 m2....,Maison,7,185,78170


In [10]:
#remove ventes
out = out[out["Bien"].str.contains("Vente") == False]

In [11]:
new_df = out[['Bien', 'Pieces', 'Taille', 'Prix', 'Ville', 'Code_Postal', 'Annonceur', 'Description']]

In [12]:
new_df.head()

,Bien,Pieces,Taille,Prix,Ville,Code_Postal,Annonceur,Description
0,Appartement,1,35,260 000,Alfortville,94140,Professionnel,Superbe studio spacieux et fonctionnel hyper c...
1,Maison,3,66,140 000,Beton-Bazoches,77320,Professionnel,Maison familiale 3 pièces avec cheminée. maiso...
2,Maison,4,179,222 000,La Ferté-sous-Jouarre,77260,Professionnel,"Atypique maison briarde, logement à l'étage av..."
3,Appartement,3,70,475 000,Bezons,95870,Professionnel,Appartement 3 pièces. appartement t3 récent de...
4,Maison,7,185,1 200 000,La Celle-Saint-Cloud,78170,Professionnel,Maison la celle saint cloud 7 pièce(s) 185 m2....
